In [5]:
# Import necessary libraries and modules for the GAN implementation
from keras.datasets import mnist
from keras.layers import Input,Dense,Reshape,Flatten
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.models import Sequential,Model
# from keras.optimizers import Adam
from tensorflow.keras.optimizers.legacy import Adam
import matplotlib.pyplot as plt
import numpy as np

In [6]:
img_rows = 28  # Number of rows (height) in the image
img_cols = 28  # Number of columns (width) in the image
channels = 1   # Number of image channels (1 for grayscale, 3 for RGB)
img_shape = (img_rows, img_cols, channels)  # Shape of the image data (height, width, channels)



In [7]:
from keras.layers import ELU

def build_generator():
    noise_shape = (100,)

    model = Sequential()
    model.add(Dense(512, input_shape=noise_shape))  # Increased complexity
    model.add(ELU(alpha=1.0))  # ELU activation function
    model.add(BatchNormalization(momentum=0.9))  # Adjusted momentum
    model.add(Dense(1024))  # Increased complexity
    model.add(ELU(alpha=1.0))  # ELU activation function
    model.add(BatchNormalization(momentum=0.9))  # Adjusted momentum
    model.add(Dense(np.prod(img_shape), activation='tanh'))  # tanh activation
    model.add(Reshape(img_shape))
    model.summary()
    noise = Input(shape=noise_shape)
    img = model(noise)
    return Model(noise, img)

In [8]:
def build_dis():
    model = Sequential()
    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))  # Increased complexity
    model.add(ELU(alpha=1.0))  # ELU activation function
    model.add(Dense(256))  # Increased complexity
    model.add(ELU(alpha=1.0))  # ELU activation function
    model.add(Dense(1, activation='tanh'))  # tanh activation
    model.summary()

    img = Input(shape=img_shape)
    validity = model(img)
    return Model(img, validity)

In [9]:
from tensorflow.keras.optimizers.schedules import ExponentialDecay

# Learning rate schedule
initial_learning_rate = 0.01
lr_schedule = ExponentialDecay(
    initial_learning_rate, decay_steps=100, decay_rate=0.96, staircase=True
)
optimizer = Adam(learning_rate=lr_schedule)  # Update the optimizer

def train(epochs, batch_size=128, save_interval=50):
    (X_train, _), (_, _) = mnist.load_data()
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    X_train = np.expand_dims(X_train, axis=3)
    half_batch = int(batch_size / 2)

    for epoch in range(epochs):
        idx = np.random.randint(0, X_train.shape[0], half_batch)
        imgs = X_train[idx]
        noise = np.random.normal(0, 1, (half_batch, 100))

        gen_imgs = generator.predict(noise)

        d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
        d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))

        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        noise = np.random.normal(0, 1, (batch_size, 100))
        valid_y = np.array([1] * batch_size)

        g_loss = combined.train_on_batch(noise, valid_y)

        print(f"{epoch} [D loss: {d_loss[0]:.6f}, acc: {100 * d_loss[1]:.2f}%] [G loss: {g_loss:.6f}]")

        if epoch % save_interval == 0:
            save_imgs(epoch)

        # # Early stopping if G loss increases
        # if epoch > 0 and g_loss > prev_g_loss:
        #     print("Early stopping due to increasing G loss.")
        #     break
        # prev_g_loss = g_loss


In [10]:
#save images after the entered interval
def save_imgs(epoch):
  r,c=5,5
  noise=np.random.normal(0,1,(r*c,100))
  gen_imgs=generator.predict(noise)


  #rescale
  gen_imgs=0.5 *gen_imgs+.5
  fig,axs=plt.subplots(r,c)
  cnt=0
  for i in range(r):
    for j in range(c):
      axs[i,j].imshow(gen_imgs[cnt,:,:,0],cmap='gray')
      axs[i,j].axis('off')
      cnt+=1
  fig.savefig(r"C:\Users\dhruv\OneDrive\Desktop\images2\minst_%d.png" % epoch)
  plt.close()

In [11]:
from keras.optimizers import RMSprop

# Define a new optimizer
optimizer = RMSprop(learning_rate=0.00005)

# Create the discriminator with adjusted optimizer
discriminator = build_dis()
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 512)               401920    
                                                                 
 elu (ELU)                   (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 elu_1 (ELU)                 (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 257       
                                                                 
Total params: 533,505
Trainable params: 533,505
Non-trai

In [12]:
generator=build_generator()
generator.compile(loss='binary_crossentropy',optimizer=optimizer)

z=Input(shape=(100,))
img=generator(z)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 512)               51712     
                                                                 
 elu_2 (ELU)                 (None, 512)               0         
                                                                 
 batch_normalization (BatchN  (None, 512)              2048      
 ormalization)                                                   
                                                                 
 dense_4 (Dense)             (None, 1024)              525312    
                                                                 
 elu_3 (ELU)                 (None, 1024)              0         
                                                                 
 batch_normalization_1 (Batc  (None, 1024)             4096      
 hNormalization)                                      

In [13]:
discriminator.trainable=False

valid=discriminator(img)

combined=Model(z,valid)
combined.compile(loss='binary_crossentropy',optimizer=optimizer)




In [14]:
#train function implemented here
train(epochs=13000,batch_size=32,save_interval=1000)

1/1 [==============================] - 0s 156ms/step
0 [D loss: 6.934238, acc: 43.75%] [G loss: 9.388892]
1/1 [==============================] - 0s 15ms/step
1 [D loss: 1.194007, acc: 84.38%] [G loss: 9.734578]
1/1 [==============================] - 0s 15ms/step
2 [D loss: 0.196368, acc: 87.50%] [G loss: 10.228333]
1/1 [==============================] - 0s 16ms/step
3 [D loss: 0.181843, acc: 90.62%] [G loss: 8.746344]
1/1 [==============================] - 0s 19ms/step
4 [D loss: 0.188742, acc: 90.62%] [G loss: 9.445855]
1/1 [==============================] - 0s 18ms/step
5 [D loss: 0.135107, acc: 93.75%] [G loss: 9.094925]
1/1 [==============================] - 0s 20ms/step
6 [D loss: 0.023805, acc: 100.00%] [G loss: 10.407704]
1/1 [==============================] - 0s 16ms/step
7 [D loss: 0.170676, acc: 90.62%] [G loss: 10.891599]
1/1 [==============================] - 0s 16ms/step
8 [D loss: 0.048755, acc: 96.88%] [G loss: 9.781248]
1/1 [==============================] - 0s 19ms/ste

In [15]:
#save the h5 file of the model.
generator.save('generator_model2.h5')
generator.save(r"C:\Users\dhruv\OneDrive\Desktop\images2\generator_model.h5")
